In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [121]:
df = pd.read_csv('C:/Users/deosd/OneDrive/Documents/HR.csv')

In [123]:
#df.shape

df.head()

,city,name,postalCode,province,reviews.rating,reviews.text,reviews.title
0,Mableton,Hotel Russo Palace,30126,GA,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds
1,Mableton,Hotel Russo Palace,30126,GA,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!
2,Mableton,Hotel Russo Palace,30126,GA,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge
3,Mableton,Hotel Russo Palace,30126,GA,5.0,We stayed here for four nights in October. The...,Good location on the Lido.
4,Mableton,Hotel Russo Palace,30126,GA,5.0,We stayed here for four nights in October. The...,������ ���������������


In [131]:
print(df.describe())

       reviews.rating
count    35050.000000
mean         3.776431
std          1.416195
min          0.000000
25%          3.000000
50%          4.000000
75%          5.000000
max         10.000000


In [135]:
# Check for missing values in each column
missing_data = df.isnull().sum()

print(missing_data)

city              0
name              0
postalCode        0
province          0
reviews.rating    0
dtype: int64


In [129]:
# Drop columns by name
df.drop(columns=['reviews.title', 'reviews.text'], inplace=True)

print(df.head())

       city                name postalCode province  reviews.rating
0  Mableton  Hotel Russo Palace      30126       GA             4.0
1  Mableton  Hotel Russo Palace      30126       GA             5.0
2  Mableton  Hotel Russo Palace      30126       GA             5.0
3  Mableton  Hotel Russo Palace      30126       GA             5.0
4  Mableton  Hotel Russo Palace      30126       GA             5.0


In [133]:
#remove missing 
df = df.dropna()

In [137]:
df.shape

(34995, 5)

In [102]:
#df.count()

In [139]:
#concatenate
df['location'] = df['city'] + ', ' + df['province'] + ' ' + df['postalCode']
#print(df)

In [ ]:
# Create the plot
sns.displot(df['reviews.rating'], kde=False)

# Apply log scale to the y-axis to make counts more readable
plt.yscale('log')

# Show the plot
plt.show()

In [74]:
hotels_df = df

In [100]:
#hotels_df['']

In [96]:
from sklearn.metrics.pairwise import cosine_similarity
from urllib.parse import quote_plus

# If 'review_count' does not exist in the DataFrame, generate random values
if 'review_count' not in hotels_df.columns:
    # Generate random review counts matching the length of the DataFrame
    np.random.seed(42)  # For reproducibility
    hotels_df['review_count'] = np.random.randint(50, 500, size=len(hotels_df))

# Step 1: Aggregate ratings for each hotel and handle location
aggregated_hotels_df = hotels_df.groupby('name').agg(
    average_rating=('reviews.rating', 'mean'),
    location=('location', lambda x: x.mode()[0])  # Use the most frequent location
).reset_index()

# Step 2: Add 'review_count' column (sum of reviews for each hotel)
aggregated_hotels_df['review_count'] = hotels_df.groupby('name')['review_count'].sum().values

# Step 3: Add variance in ratings as a feature to calculate similarity better
aggregated_hotels_df['rating_variance'] = hotels_df.groupby('name')['reviews.rating'].var().values

# Step 4: Normalize features (ratings, review count, and rating variance) to make them comparable
aggregated_hotels_df['normalized_rating'] = (aggregated_hotels_df['average_rating'] - aggregated_hotels_df['average_rating'].mean()) / aggregated_hotels_df['average_rating'].std()
aggregated_hotels_df['normalized_review_count'] = (aggregated_hotels_df['review_count'] - aggregated_hotels_df['review_count'].mean()) / aggregated_hotels_df['review_count'].std()
aggregated_hotels_df['normalized_rating_variance'] = (aggregated_hotels_df['rating_variance'] - aggregated_hotels_df['rating_variance'].mean()) / aggregated_hotels_df['rating_variance'].std()

# Step 5: Handle missing (NaN) values: Drop rows with NaN or fill with mean/median
aggregated_hotels_df = aggregated_hotels_df.dropna()  # Option 1: Drop rows with NaN values

# Alternatively, you could fill NaNs with a default value like the mean or median:
# aggregated_hotels_df = aggregated_hotels_df.fillna(aggregated_hotels_df.mean())  # Option 2: Fill NaNs with column mean

# Combine normalized features into one matrix
features_matrix = aggregated_hotels_df[['normalized_rating', 'normalized_review_count', 'normalized_rating_variance']].values

# Step 6: Build recommendations based on cosine similarity of these features
def build_hotel_recommendations(hotel_name):
    try:
        hotel_name = hotel_name.lower()

        # Find the hotel index from the DataFrame (using aggregated data now)
        try:
            idx = aggregated_hotels_df[aggregated_hotels_df['name'].apply(lambda x: x.lower()) == hotel_name].index[0]
        except IndexError: 
            return None  # If the hotel is not found in the list

        # Compute similarity using multiple features (normalized ratings, review count, and variance)
        cosine_sim = cosine_similarity(features_matrix[idx].reshape(1, -1), features_matrix).flatten()

        # Get the top 5 most similar hotels (excluding the input hotel itself)
        similar_hotels_idx = cosine_sim.argsort()[::-1][1:6]  # Exclude the input hotel by slicing from 1 onwards

        # Extract hotel information for the recommended hotels
        recommendations = aggregated_hotels_df.iloc[similar_hotels_idx][['name', 'average_rating', 'location', 'review_count']]
        recommendations['similarity'] = cosine_sim[similar_hotels_idx]

        return recommendations[['name', 'average_rating', 'location', 'review_count', 'similarity']]
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

# Step 7: Create recommendations with proper URLs
def get_hotel_recommendations(hotel_name):
    recommendations = build_hotel_recommendations(hotel_name)
    
    if recommendations is None:
        return recommendations
    else:
        # Create clean URLs using quote_plus to handle spaces and special characters
        recommendations['urls'] = [
            f'https://www.tripadvisor.com/Search?q={quote_plus(name)}&geo=1&ssrc=a&searchNearby=false&searchSessionId=000e238dc3e041c2.ssid&offset=0' for name in recommendations['name']
        ]
        
        # Return the final recommendations
        return recommendations[['name', 'average_rating', 'location', 'review_count', 'similarity', 'urls']]

# Example usage
hotel_name = "Hotel Russo Palace"
recommendations = get_hotel_recommendations(hotel_name)
print(recommendations)

                                                  name  average_rating  \
297  Homewood Suites By Hilton Albuquerque - Journa...        4.204082   
122                    Country Inn & Suites By Carlson        3.966102   
296                  Homewood Suites - Mall of America        4.280702   
18                    Americinn Lodge Suites Princeton        4.744681   
94                               City Express Irapuato        4.096154   

                         location  review_count  similarity  \
297         Albuquerque, NM 87109         14240    0.999400   
122  Crystal Lake, Lakewood 60014         14576    0.999241   
296         Minneapolis, MN 55425         14111    0.999134   
18            Princeton, IL 61356         11045    0.997863   
94               Mobile, AL 36670         13680    0.996894   

                                                  urls  
297  https://www.tripadvisor.com/Search?q=Homewood+...  
122  https://www.tripadvisor.com/Search?q=Country+I...  
296  

In [98]:
recommendations.to_csv('sampled.csv', index=False)